In [16]:
from sklearn import datasets

boston = datasets.load_boston()
X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]


In [17]:
from moon.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, seed=666)


# 使用传统的线性回归（正规方程）方式进行计算：

In [18]:
%%time
from moon.linear_model import MultipleLinearRegression

m = MultipleLinearRegression()
regression = m
regression.fit_normal(X_train, y_train)
print(regression.score(X_test, y_test))
print(regression.coef_)
print(regression.intercept_)

0.8129794056212774
[-1.20354261e-01  3.64423279e-02 -3.61493155e-02  5.12978140e-02
 -1.15775825e+01  3.42740062e+00 -2.32311760e-02 -1.19487594e+00
  2.60101728e-01 -1.40219119e-02 -8.35430488e-01  7.80472852e-03
 -3.80923751e-01]
34.11739972320099
Wall time: 2 ms


# 使用梯度下降的方式进行计算：

In [19]:
%%time
from moon.linear_model import MultipleLinearRegression

linear_regression = m
linear_regression.fit_gradient_descent(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
D:\PythonProject\Luna\moon\linear_model.py:97: RuntimeWarning: overflow encountered in square
  return np.sum((y - X_b.dot(theta)) ** 2) / len(X_b)
D:\PythonProject\Luna\moon\linear_model.py:116: RuntimeWarning: invalid value encountered in double_scalars
  if abs(J(last_theta, X_b, y) - J(theta, X_b, y)) < epsilon:


Wall time: 282 ms


In [20]:
linear_regression.coef_

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

根据观察，每一个测试集的特征维度并不在同一个数据量级中，所以$\eta$的取值需要非常小才能尽可能缩小这个量级差距

In [21]:
linear_regression.fit_gradient_descent(X_train,y_train,eta=.000001)

In [22]:
linear_regression.score(X_test,y_test)

0.27586818724477224

可以看到，这个$R^2$值太小，这可能是因为学习率$\eta$太小，使得人为干预的学习次数达到上限


In [23]:
%time linear_regression.fit_gradient_descent(X_train,y_train,eta=.000001,n_iters=1e6)

Wall time: 25.6 s


In [24]:
linear_regression.score(X_test,y_test)

0.7542932581943915

In [31]:
X_train[:10]

array([[1.42362e+01, 0.00000e+00, 1.81000e+01, 0.00000e+00, 6.93000e-01,
        6.34300e+00, 1.00000e+02, 1.57410e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.96900e+02, 2.03200e+01],
       [3.67822e+00, 0.00000e+00, 1.81000e+01, 0.00000e+00, 7.70000e-01,
        5.36200e+00, 9.62000e+01, 2.10360e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.80790e+02, 1.01900e+01],
       [1.04690e-01, 4.00000e+01, 6.41000e+00, 1.00000e+00, 4.47000e-01,
        7.26700e+00, 4.90000e+01, 4.78720e+00, 4.00000e+00, 2.54000e+02,
        1.76000e+01, 3.89250e+02, 6.05000e+00],
       [1.15172e+00, 0.00000e+00, 8.14000e+00, 0.00000e+00, 5.38000e-01,
        5.70100e+00, 9.50000e+01, 3.78720e+00, 4.00000e+00, 3.07000e+02,
        2.10000e+01, 3.58770e+02, 1.83500e+01],
       [6.58800e-02, 0.00000e+00, 2.46000e+00, 0.00000e+00, 4.88000e-01,
        7.76500e+00, 8.33000e+01, 2.74100e+00, 3.00000e+00, 1.93000e+02,
        1.78000e+01, 3.95560e+02, 7.56000e+00],
       [2.49800e-02, 0.00000e+

显然，在使用梯度下降法时，我们需要使用数据归一化进行计算

In [34]:
# from moon.preprocessing import StandardScaler
#
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train_std = scaler.transform(X_train)
# X_train_std[:10]

from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit(X_train)
X_train_std = std.transform(X_train)
X_train_std[:10]

array([[ 1.22372711e+00, -4.72938312e-01,  1.00449680e+00,
        -2.37915476e-01,  1.17672219e+00,  1.74619704e-01,
         1.12041788e+00, -1.04618570e+00,  1.66456428e+00,
         1.53040651e+00,  7.79319066e-01,  4.39429823e-01,
         1.01121088e+00],
       [ 2.85335963e-03, -4.72938312e-01,  1.00449680e+00,
        -2.37915476e-01,  1.83672499e+00, -1.31943651e+00,
         9.87370381e-01, -7.96657333e-01,  1.66456428e+00,
         1.53040651e+00,  7.79319066e-01,  2.62156418e-01,
        -4.03855745e-01],
       [-4.10372369e-01,  1.27996861e+00, -7.12624284e-01,
         4.20317340e+00, -9.31858180e-01,  1.58186531e+00,
        -6.65219648e-01,  4.67996723e-01, -6.36649609e-01,
        -9.32591469e-01, -4.50790880e-01,  3.55249714e-01,
        -9.82175177e-01],
       [-2.89298883e-01, -4.72938312e-01, -4.58507991e-01,
        -2.37915476e-01, -1.51854871e-01, -8.03141854e-01,
         9.45355380e-01, -3.25609248e-03, -6.36649609e-01,
        -6.15749496e-01,  1.15781443e

In [35]:
multiple_linear_regression = MultipleLinearRegression()
%time multiple_linear_regression.fit_gradient_descent(X_train_std, y_train)
print(multiple_linear_regression.coef_)
print(multiple_linear_regression.intercept_)

Wall time: 119 ms
[-1.04042202  0.83093351 -0.24794356  0.01179456 -1.35034756  2.25074
 -0.66384353 -2.53568774  2.25572406 -2.34011572 -1.76565394  0.70923397
 -2.72677064]
21.500765306122382


In [36]:
X_test_std = std.transform(X_test)

In [37]:
multiple_linear_regression.score(X_test_std,y_test)

0.8129873310487505